ID Number: 33385806

# EEG-based BCI using Emotiv Epoc X and Visual Imagery: An Exploratory Study

## MSc Project for Computational Cognitive Neuroscience 2020/2021

### In this notebook I have collected the main functions to carry the out EEG Pre-processing Analysis

### 1. Function to load the EEG datasets

The code below defines a function to load all data files with extensions '.edf':

In [1]:
def load_data(path):
  
    '''
    Load the .edf datasets
    
    Parameters
    ----------
    path : the directory path where your data are stored
 
 
    Returns
    -------
    list_load_dataset : list of .edf files
    '''
    
    
    list_files = os.listdir(path=path) #set the directory path
    
    extension = '.edf'
    index = 0
    list_dataset = [] #create an empty list to store our .edf files
    for file in list_files: #for each file in our directory
        if extension in list_files[index]: #if the file's extension is equal to .edf
            list_dataset.append(list_files[index]) #add the file in list_dataset
        index += 1 

    list_load_dataset = []
    for n_file in range(0, len(list_dataset)): #for each .edf file in our list 
        dataset = read_raw_edf(list_dataset[n_file], preload=True) #load the file
        list_load_dataset.append(dataset)
        
    return list_load_dataset

### 2. Function to exclude unused channnels

The function below iterate thorugh each .edf file and remove the unwanted channels (i.e. the channels excluded from the include_channels list):

In [2]:
def excl_chan(data):

    ''' 
    This function exclude the channels we don't need from further analysis. If you want
    to add or remove some channels, modify the above list "include_channels".
    
    Parameters
    ----------
    data: our raw datasets
    
    Returns
    -------
    list_datasets: the list of datasets with unused channels removed
    
    '''
       
    list_datasets=[]     
    for n_file in range(0, len(data)):    
         for chan_name in data[n_file].ch_names: 
            if chan_name not in include_channels:
                data[n_file].drop_channels([chan_name])   
                list_datasets.append(data)
                
                                                   
    return list_datasets

### 3. Filtering datasets

The following function iterates through each file and apply a Hamming windowed FIR band-pass filter (default):

In [3]:
 def filter_data(data):
    
    '''
    This function filter the raw datasets. Because we are interested in low frequencies, 
    in the alpha-beta frequency range, we can band-pass filter between 1Hz-40Hz.
    
    Parameters
    ----------
    data: our raw continuous unfiltered datasets
    
    Returns
    -------
    filtered_data: the datasets containing the filtered data.
    '''


    filtered_data=[]
    for file in range(0, len(data)):
        data[file].filter(1., 30., fir_design='firwin') #apply band-pass filter between 1 and 40 HZ, our freqs range of interest
        filtered_data.append(data[file])
        
    return filtered_data
        

### 4. Function to create epochs

The function below is used to epoch the continuous filtered EEG datasets to segments with duration 6.998 seconds.
With 30 sessions, we will have 300 epochs in total. 

In [1]:
def make_epochs(list_prep_dataset, duration):
    
    """ 
    This function extract the epochs object from the preprocessed list of datasets. The conditions are:
    Push = 1
    Relax = 0
    --> Note: This function should be called after the preprocessing, but before the ICA <--

    :param list_prep_dataset: a list containing the preprocessed datasets
    :param duration: the duration of each epochs (10 seconds)
    :return: epochs: the mne.Epochs object containing the epoched data.
    """
    
    event_dict = {'Relax': 0, 'Push': 1}   
 
    list_epochs = []  
    for prep_dataset in list_prep_dataset:
        events = mne.make_fixed_length_events(prep_dataset, id=0, start=65.0, stop=165.0, duration=duration) # make fixed-length events for each dataset in list_prep_dateset
        
        for n_events in range(0, len(events)):
            if n_events % 2 == 1: 
                events[n_events][2] = 1 
                
          
        # make epochs for each dataset
        epochs = mne.Epochs(prep_dataset, events, tmin=0.0, tmax=9.998, event_id=event_dict, baseline=(0, 0), preload=True)
 
        list_epochs.append(epochs)
                
        # combine epochs
        epochs = mne.concatenate_epochs(list_epochs) 

        # crop start and end of the epochs based on provided time reference
        epochs.crop(tmin=0.25, tmax=9.998 - 0.25) 
              
        # Generate Standard montage (useful for ICA and TimeFrequency analyses)
        biosemi_montage = mne.channels.make_standard_montage('standard_1020')
        epochs.set_montage(biosemi_montage) 
        
    return epochs

### 5. Function to plot Power Spectrum Density (PSD)


Function to plot the PSD for each session to visually inspect them:


In [5]:
def plot_data(data):   #the input is, for example, our filtered dataset
    for file in range(0, len(data)):
        data[file].plot_psd(average=True)   


### 6. Function to compute Morlet wavelets on epochs (to feed to the Common Spatial Pattern technique)


In [ ]:
#This function must be used only with the CSP function provided from MNE toolbox

def epochs_power(data): #our input data are the epoched data
    
    #apply Morlet wavelets
    freqs=np.ravel(freq_ranges)
    pw = mne.time_frequency.tfr_morlet(data, freqs=freqs, n_cycles=n_cycles, use_fft=True, average=False,return_itc=False, decim=3, n_jobs=1)
    n_col= pw.data.shape[3] #extract this dimension to use later 
    n_chan=pw.data.shape[1]
    n_row= pw.data.shape[0]
    
    #average power in the freq band of interest
    epo_pw = np.zeros(shape=(n_row,n_chan,n_col))  #initialise the variable with dimensions previously extracted
    counter=0
    
    for samples in range (0,n_row):
        for chan in range(0, n_chan): 
            select_pw = pw.data[samples][chan][(pw.freqs>fmin) & (pw.freqs<=fmax)][:]  
            counter+=1
            pw_avg = np.mean(select_pw, axis=0)  
            epo_pw[samples,chan,:]=pw_avg
    return epo_pw #the output is the average across all frequencies within each band, with dimensions (n_row,n_chan,n_col)
